In [18]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip \
    -O /tmp/horse-or-human.zip

--2020-04-06 09:53:46--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.216.128, 2607:f8b0:400c:c07::80
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.216.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘/tmp/horse-or-human.zip’

/tmp/horse-or-human 100%[===================>] 142.65M   156MB/s    in 0.9s    

2020-04-06 09:53:47 (156 MB/s) - ‘/tmp/horse-or-human.zip’ saved [149574867/149574867]



In [0]:
import os
import zipfile

local_zip = '/tmp/horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/horse-or-human')
zip_ref.close()

In [20]:
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epoch,log={}):
        if(log.get('accuracy')>0.999):
            print("\nReached 99% accuracy, stopping the training")
            self.model.stop_training=True
callbacks = myCallback()
model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512,activation='relu'),
        tf.keras.layers.Dense(1,activation='sigmoid')
       
    ])
model.compile(loss='binary_crossentropy',optimizer=RMSprop(lr=0.001),metrics=['accuracy'])   
train_gen = ImageDataGenerator(rescale=1.0/255)
train_gendat= train_gen.flow_from_directory("/tmp/horse-or-human",target_size=(150,150),batch_size=10,class_mode='binary')
history=model.fit_generator(train_gendat,steps_per_epoch=2,epochs=20,verbose=1,callbacks=[callbacks])

Found 1027 images belonging to 2 classes.
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/20
2/2 [==============================] - 0s 185ms/step - loss: 5.2654 - accuracy: 0.5500
Epoch 2/20
2/2 [==============================] - 0s 160ms/step - loss: 1.6013 - accuracy: 0.3500
Epoch 3/20
2/2 [==============================] - 0s 153ms/step - loss: 0.8389 - accuracy: 0.4000
Epoch 4/20
2/2 [==============================] - 0s 156ms/step - loss: 0.6246 - accuracy: 0.6000
Epoch 5/20
2/2 [==============================] - 0s 155ms/step - loss: 0.6669 - accuracy: 0.5000
Epoch 6/20
2/2 [==============================] - 0s 160ms/step - loss: 0.4570 - accuracy: 0.8500
Epoch 7/20
2/2 [==============================] - 0s 176ms/step - loss: 0.7206 - accuracy: 0.8000
Epoch 8/20
2/2 [==============================] - 0s 160ms/step - loss: 0.5177 - accuracy: 0.7500
Epoch 9/20
2/2 [==============================] - 0s 160ms/step - loss: 0.4288 - accuracy: 0.9000


In [22]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(150, 150))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is a human")
  else:
    print(fn + " is a horse")

Saving 1c39a2f32b2c117af2380c60414739be.png to 1c39a2f32b2c117af2380c60414739be (1).png
[1.]
1c39a2f32b2c117af2380c60414739be.png is a human
